In [16]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append(2*'../')

import os
import glob
from pathlib import Path
from omegaconf import DictConfig
import yaml

import torch
import lightning as L

from rl4co.tasks.rl4co import RL4COLitModule
from rl4co.utils.lightning import load_model_from_checkpoint


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading model from checkpoint

In [17]:
checkpoints_path = Path('../../saved_checkpoints/')

exp_name = 'tsp50'
model_name = 'am-tsp50'
checkpoints_path = checkpoints_path / exp_name / model_name

cfg_path = checkpoints_path / 'config.yaml'
ckpt_path = checkpoints_path / 'epoch_099.ckpt'

lit_module = load_model_from_checkpoint(cfg_path, ckpt_path, phase='test')

Unused kwargs: {'params': {'total': 708608, 'trainable': 708608, 'non_trainable': 0}}


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

policy = lit_module.model.policy.to(device)
policy.eval()
env = lit_module.model.env

## Simple greedy eval

In [19]:

test_dataset = lit_module.test_dataset
dataloader = lit_module._dataloader(test_dataset, batch_size=512)

with torch.no_grad():

    rewards = []

    for batch in dataloader:
        td = batch.to(device).clone()
        td = env.reset(td)
        out = policy(td, decode_type="greedy")
        rewards.append(out['reward'])

    rewards = torch.cat(rewards)
    print(rewards.mean())

tensor(-5.7785, device='cuda:0')
